# 1. SETTINGS

In [0]:
# settings for Google Colab
#from google.colab import drive
#drive.mount('/gdrive')
import os
os.chdir("/gdrive/My Drive/Colab Notebooks/DMC_2019/codes")

In [0]:
# import packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA, FastICA, FactorAnalysis
from sklearn.metrics import log_loss
from sklearn.neighbors import NearestNeighbors
import copy
import scipy.stats
import os
import time

In [0]:
# helper functions
import functions
from functions import *

In [0]:
# pandas options
pd.set_option('display.max_columns', None)

In [0]:
# dark background style
plt.style.use('dark_background')

In [0]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [0]:
# garbage collection
import gc
gc.enable()

# 2. DATA PREPARATION

In [24]:
# import CSV
df = pd.read_csv('../data/data_v3.csv')
print(df.shape)

(500000, 13)


In [0]:
# target variable
target = 'fraud'

In [26]:
# partitioning
train = df[df[target].isnull() == False]
test  = df[df[target].isnull() == True]
print(train.shape)
print(test.shape)

(1879, 13)
(498121, 13)


In [27]:
# create neigbhor-based featurs (by Liza)

'''features_for_n = ['grandTotal',
 'lineItemVoids',
 'lineItemVoidsPerPosition',
 'quantityModifications',
 'scannedLineItemsPerSecond',
 'scansWithoutRegistration',
 'totalScanTimeInSeconds',
 'valuePerSecond',
 'total_items',
 'weird_actions']

train['neighbors_fraud'] = 0
test['neighbors_fraud'] = 0

print('create train neighboring features')
nbrs = NearestNeighbors(n_neighbors=100, algorithm='ball_tree').fit(train.loc[:,features_for_n].values)
for i, obs in enumerate(train.index):
  distances, indices = nbrs.kneighbors(train.loc[[obs], features_for_n].values)
  train.loc[obs, 'neighbors_fraud'] = train.loc[indices[0][indices[0]!=obs], 'fraud'].mean()
  if i%100==0:
    print(i)'''

    
    
#print('create train neighboring features')
#nbrs = NearestNeighbors(n_neighbors=16, algorithm='ball_tree').fit(train.loc[:,features_for_n])
#for i, obs in enumerate(train.index):
#  distances, indices = nbrs.kneighbors(train.loc[[obs], features_for_n])
#  train.loc[obs, 'neighbors_fraud15'] = train.loc[indices[0][indices[0]!=obs], 'fraud'].mean()
#  if i%100==0:
#    print(i)
    
#print('create test neighboring features')  
#nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(train.loc[:,features_for_n])
#for i, obs in enumerate(test.index):
#  distances, indices = nbrs.kneighbors(test.loc[[obs], features_for_n])
#  test.loc[obs, 'neighbors_fraud'] = train.loc[indices[0], 'fraud'].mean()
#  if i%1000==0:
#    print(i)

"features_for_n = ['grandTotal',\n 'lineItemVoids',\n 'lineItemVoidsPerPosition',\n 'quantityModifications',\n 'scannedLineItemsPerSecond',\n 'scansWithoutRegistration',\n 'totalScanTimeInSeconds',\n 'valuePerSecond',\n 'total_items',\n 'weird_actions']\n\ntrain['neighbors_fraud'] = 0\ntest['neighbors_fraud'] = 0\n\nprint('create train neighboring features')\nnbrs = NearestNeighbors(n_neighbors=100, algorithm='ball_tree').fit(train.loc[:,features_for_n].values)\nfor i, obs in enumerate(train.index):\n  distances, indices = nbrs.kneighbors(train.loc[[obs], features_for_n].values)\n  train.loc[obs, 'neighbors_fraud'] = train.loc[indices[0][indices[0]!=obs], 'fraud'].mean()\n  if i%100==0:\n    print(i)"

In [0]:
# target variable
y = train[target]
del train[target], test[target]

# 3. MODELING

### PARAMETERS

In [29]:
# drop bad features
excluded_feats = ['id']
features = [f for f in train.columns if f not in excluded_feats]
print(train[features].shape)

(1879, 11)


In [0]:
# perform PCA
#num_comp = 5
#pca = PCA(n_components = num_comp)
#pca.fit(train[features].values)
#train = pd.DataFrame(pca.transform(train[features].values), columns = ['v' + str(v) for v in range(0, num_comp)])
#test  = pd.DataFrame(pca.transform(test[features].values),  columns = ['v' + str(v) for v in range(0, num_comp)])
#features = list(train.columns)
#print(train[features].shape)
#print(test[features].shape)

In [0]:
### PARAMETERS

# settings
cores = 12
seed  = 23

# cross-validation
num_folds = 5
shuffle   = True

# muner of rounds
max_rounds = 600
stopping   = 600
verbose    = 200

# LGB parameters
lgb_params = {
    'boosting_type':     'gbdt',
    'objective':         'binary',
    'metrics':           'binary_logloss',
    'bagging_fraction':  0.9,
    'feature_fraction':  0.8,
    'lambda_l1':         0.1,
    'lambda_l2':         0.1,
    'min_split_gain':    0.01,
    'min_child_weight':  2,
    'min_child_samples': 20,
    'silent':            True,
    'verbosity':         -1,
    'learning_rate':     0.1,
    'max_depth':         7,
    'num_leaves':        70,
    'scale_pos_weight':  1,
    'n_estimators':      max_rounds,
    'nthread' :          cores,
    'random_state':      seed,
}

# data partitinoing
folds = StratifiedKFold(n_splits = num_folds, random_state = seed, shuffle = shuffle)

In [0]:
# placeholders
clfs = []
valid_profit = np.zeros(num_folds) 
preds_test   = np.zeros(test.shape[0])
preds_oof    = np.zeros(train.shape[0])
importances  = pd.DataFrame()

In [0]:
# SMOTE settings
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 23, n_jobs = 10, sampling_strategy = 0.1)

### CROSS-VALIDATION

In [34]:
### CROSS-VALIDATION LOOP
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train, y)):
    
    # data partitioning
    trn_x, trn_y = train[features].iloc[trn_idx], y.iloc[trn_idx]
    val_x, val_y = train[features].iloc[val_idx], y.iloc[val_idx]
    
    ## remove outliers
    #out_idx = (np.abs(scipy.stats.zscore(trn_x)) < 10).all(axis = 1)
    #trn_x = trn_x[out_idx]
    #trn_y = trn_y[out_idx]
    
    # scale data
    scaler   = RobustScaler()
    trn_x    = pd.DataFrame(scaler.fit_transform(trn_x),      columns = features)
    val_x    = pd.DataFrame(scaler.transform(val_x),          columns = features)
    tmp_test = pd.DataFrame(scaler.transform(test[features]), columns = features)

    # augment training data with SMOTE
    trn_x, trn_y = sm.fit_sample(trn_x, trn_y)
    trn_x = pd.DataFrame(trn_x, columns = features)
    trn_y = pd.Series(trn_y)
    
    # factor decomposition
    tmp_features = copy.deepcopy(features)
    #if False:
    #    decomp = FactorAnalysis(n_components = 11)
    #    decomp.fit(trn_x)
    #    trn_x = decomp.transform(trn_x)
    #    val_x = decomp.transform(val_x)
    #    tmp_test = decomp.transform(tmp_test)
    #    tmp_features = ['pc'+str(i) for i in range(decomp.n_components)]
    
    # add noise to train to reduce overfitting
    #trn_x += np.random.normal(0, 0.01, trn_x.shape)
    
    # mean target encoding
    #trn_x, val_x, tmp_test = mean_target_encoding(trn_x, val_x, test, features = ['trustLevel'], target = 'fraud', folds = 5)
    #features = [f for f in trn_x.columns if f not in excluded_feats]
        
    # train lightGBM
    print('Custom early stopping: select the best out of %.0f iterations...' % max_rounds)
    clf = lgb.LGBMClassifier(**lgb_params) 
    clf = clf.fit(trn_x, trn_y, 
                  eval_set              = [(trn_x, trn_y), (val_x, val_y)], 
                  eval_metric           = prediction_reward, 
                  #eval_metric           = "logloss", 
                  #early_stopping_rounds = stopping,
                  verbose               = verbose)
    clfs.append(clf)
    
    # predict validation from the best iteration
    #best_iter = clf.best_iteration_
    best_iter = np.argmax(clf.evals_result_['valid_1']['profit']) + 1
    val_preds = clf.predict_proba(val_x, num_iteration = best_iter)[:, 1]
    print('Best iteration is:')
    print('[' + str(best_iter) + ']   valid_1 profit: ' + str(prediction_reward(val_y, val_preds)[1].astype('int')) + 
          "; log_loss = ", str(np.round(log_loss(val_y, val_preds), 6)))
       
    
############ PERFORM 1 ITERATRION OF SELF-TRAINING
#
#    # predict unlabeled data
#    tmp_preds_test = clf.predict_proba(tmp_test, num_iteration = best_iter)[:, 1]
#
#    # extract most confident preds
#    perc_ones  = 0.0001
#    perc_zeros = 0.0001
#    pmax = np.quantile(tmp_preds_test, 1 - perc_ones)
#    pmin = np.quantile(tmp_preds_test, perc_zeros)
#    tmp_preds_test[tmp_preds_test >= pmax] = 1
#    tmp_preds_test[tmp_preds_test <= pmin] = 0
#    confident_ones  = np.where(tmp_preds_test == 1)
#    confident_zeros = np.where(tmp_preds_test == 0)
#    confident_ones  = confident_ones[0][0:np.round(len(tmp_test)  * perc_ones).astype('int')]
#    confident_zeros = confident_zeros[0][0:np.round(len(tmp_test) * perc_zeros).astype('int')]
#    confident_idx   = np.concatenate((confident_ones, confident_zeros))
#
#    # append new data to train
#    test_X = tmp_test.iloc[confident_idx]
#    test_y = pd.Series(tmp_preds_test[confident_idx])
#    trn_x  = trn_x.append(test_X)
#    trn_y  = trn_y.append(test_y)
#    print('--------------------------------')
#    print('Added %.0f cases to training data...' % len(test_y))
#    print('--------------------------------')
#
#    # retrain lightGBM
#    print('Custom early stopping: select the best out of %.0f iterations...' % max_rounds)
#    clf = lgb.LGBMClassifier(**lgb_params) 
#    clf = clf.fit(trn_x, trn_y, 
#                  eval_set              = [(trn_x, trn_y), (val_x, val_y)], 
#                  eval_metric           = prediction_reward, 
#                  verbose               = verbose)
#
#    # find the best iteration
#    best_iter = np.argmax(clf.evals_result_['valid_1']['profit']) + 1
#    print('Best iteration is:')
#    print('[' + str(best_iter) + ']   valid_1 profit: ' + 
#          str(prediction_reward(val_y, clf.predict_proba(val_x, num_iteration = best_iter)[:, 1])[1].astype('int')) + 
#          "; log_loss = ", str(np.round(log_loss(val_y, clf.predict_proba(val_x, num_iteration = best_iter)[:, 1]), 6)))
#    
############ PERFORM 1 ITERATRION OF SELF-TRAINING


    # save predictions
    preds_oof[val_idx]    = clf.predict_proba(val_x, num_iteration = best_iter)[:, 1]
    valid_profit[n_fold]  = prediction_reward(val_y, preds_oof[val_idx])[1]
    preds_test           += clf.predict_proba(tmp_test, num_iteration = best_iter)[:, 1] / folds.n_splits 

    ## importance
    fold_importance_df = pd.DataFrame()
    fold_importance_df['Feature'] = tmp_features
    fold_importance_df['Importance'] = clf.feature_importances_
    fold_importance_df['Fold'] = n_fold + 1
    importances = pd.concat([importances, fold_importance_df], axis = 0)
    
    # print performance
    print('--------------------------------')
    print('FOLD%2d: PROFIT = %.0f' % (n_fold + 1, valid_profit[n_fold]))
    print('--------------------------------')
    print('')
        
    # clear memory
    del trn_x, trn_y, val_x, val_y
    gc.collect()
    
    # uncomment for mean target encoding
    #features = [f for f in train.columns if f not in excluded_feats]
    
    
# print overall performance    
cv_perf = np.sum(valid_profit)
print('--------------------------------')
print('TOTAL PROFIT = %.0f' % cv_perf)
print('--------------------------------')

Custom early stopping: select the best out of 600 iterations...
[200]	training's binary_logloss: 0.0057359	training's profit: 710	valid_1's binary_logloss: 0.0152655	valid_1's profit: 95
[400]	training's binary_logloss: 0.00468115	training's profit: 710	valid_1's binary_logloss: 0.0139562	valid_1's profit: 95
[600]	training's binary_logloss: 0.00461785	training's profit: 710	valid_1's binary_logloss: 0.0138809	valid_1's profit: 95
Best iteration is:
[29]   valid_1 profit: 95; log_loss =  0.043356
--------------------------------
FOLD 1: PROFIT = 95
--------------------------------

Custom early stopping: select the best out of 600 iterations...
[200]	training's binary_logloss: 0.00525732	training's profit: 710	valid_1's binary_logloss: 0.0392173	valid_1's profit: 20
[400]	training's binary_logloss: 0.00440394	training's profit: 710	valid_1's binary_logloss: 0.0406726	valid_1's profit: 55
[600]	training's binary_logloss: 0.00427252	training's profit: 710	valid_1's binary_logloss: 0.0411

In [0]:
##### RECHECK PROFIT  
prediction_reward(y, preds_oof)


###### TRACKING RESULTS (5 folds, strat = True, seed = 23)

# V1: lgb, 5 folds, default features:   80
# V2: add feature:  no. total items:   250
# V3: use logloss for ES, not profit:  260
# V4: add feature: no. weird actions:  275
# V5: custom earlystop for profit:     320
# V6: add SMOTE for minority class:    335
# V7: add robust data scaling:         350 = 95 + 55 + 75 + 35 + 90
# V8: increase learning rate to 0.1:   375 = 95 + 65 + 75 + 50 + 90

In [0]:
##### VARIABLE IMPORTANCE

# load importance    
top_feats = 100
cols = importances[['Feature', 'Importance']].groupby('Feature').mean().sort_values(by = 'Importance', ascending = False)[0:top_feats].index
importance = importances.loc[importances.Feature.isin(cols)]
    
# plot variable importance
plt.figure(figsize = (10, 6))
sns.barplot(x = 'Importance', y = 'Feature', data = importance.sort_values(by = 'Importance', ascending = False))
plt.tight_layout()

# save plot as pdf
plt.savefig('../var_importance.pdf')

### CUTOFF OPTIMIZATION

In [0]:
##### OPTIMIZE CUTOFF

# set step
step = 100

# search
cutoffs = []
profits = []
for i in range(0, step):
    cutoffs.append(i / step)
    profits.append(recompute_reward(y, preds_oof, cutoff = cutoffs[i]))
        
# results
plt.figure(figsize = (10,4))
sns.lineplot(x = cutoffs[10:step], y = profits[10:step], color = 'red')
plt.tight_layout()
plt.axvline(x = cutoffs[np.argmax(profits)], color = 'white', linestyle = '--')
print('- optimal cutoff = %.4f' % cutoffs[np.argmax(profits)])
print('- optimal profit = %.4f' % profits[np.argmax(profits)])
plt.savefig('../cutoff_selection.pdf')

# update performance
cv_perf = recompute_reward(y, preds_oof, cutoff = cutoffs[np.argmax(profits)])

# 4. SUBMISSION

In [0]:
# file name
model = 'lgb_v8'
perf  = str(round(cv_perf, 0).astype('int'))
name  = model + '_' + perf

In [0]:
# export OOF preds
oof = pd.DataFrame({'id': train['id'], 'fraud': preds_oof})
oof.to_csv('../oof_preds/' + str(name) + '.csv', index = False)
oof.head()

In [0]:
# check submission
sub = pd.DataFrame({'id': test['id'], 'fraud': preds_test})
sub['fraud'][sub['fraud'] >  cutoffs[np.argmax(profits)]] = 1
sub['fraud'][sub['fraud'] <= cutoffs[np.argmax(profits)]] = 0
sub['fraud'] = sub['fraud'].astype('int')
sub.head()

In [0]:
# export submission
sub = sub[['fraud']]
sub.to_csv('../submissions/' + str(name) + '.csv', index = False)
sub.shape

In [0]:
# check correlation with previous submission
prev_sub = pd.read_csv('../submissions/lgb_v8_375.csv')
cor = np.sum(prev_sub[target] == sub.reset_index()[target]) / len(sub)
print("Share of the same predictions: " + str(np.round(cor, 6)))